In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
import numpy as np


In [17]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = []
        self.labels = []

        for file_name in os.listdir(folder_path):
            if file_name.startswith('cat'):
                label = 0
            elif file_name.startswith('dog'):
                label = 1
            else:
                continue
            
            self.images.append(file_name)
            self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.images[idx])
        image = Image.open(img_name)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [18]:
def train_cnn_model():
    # 实例化模型
    model = define_cnn_model()
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    train_it = datagen.flow_from_directory(
        "dogsandcats/",
        class_mode="binary",
        batch_size=64,
        target_size=(200, 200))  
    # 大炼钢铁！
    model.fit_generator(train_it,
                        steps_per_epoch=len(train_it),
                        epochs=5,
                        verbose=1)
    model.save("my_model.h5")

In [19]:
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [20]:
dataset = CatsAndDogsDataset(folder_path='./dogsandcats', transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [21]:
model = models.resnet18(pretrained=True)
# 替换最后一层，全连接层输出2个类
model.fc = torch.nn.Linear(model.fc.in_features, 2)  

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# 看看设备 别用cpu!
print(device) 

cuda


In [23]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [24]:
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return torch.sum(preds == labels).item() / len(labels)

In [25]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0
    
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        total_samples += labels.size(0)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_corrects += calculate_accuracy(outputs, labels) * labels.size(0)

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = running_corrects / total_samples
    # 十步就能到99，loss0.02，不用再多了
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

print('Finished Training')


Epoch [1/10], Loss: 0.1435, Accuracy: 0.9440
Epoch [2/10], Loss: 0.0840, Accuracy: 0.9666
Epoch [3/10], Loss: 0.0703, Accuracy: 0.9717
Epoch [4/10], Loss: 0.0594, Accuracy: 0.9764
Epoch [5/10], Loss: 0.0405, Accuracy: 0.9843
Epoch [6/10], Loss: 0.0430, Accuracy: 0.9828
Epoch [7/10], Loss: 0.0331, Accuracy: 0.9882
Epoch [8/10], Loss: 0.0348, Accuracy: 0.9863
Epoch [9/10], Loss: 0.0280, Accuracy: 0.9896
Epoch [10/10], Loss: 0.0207, Accuracy: 0.9922
Finished Training


In [26]:
torch.save(model.state_dict(), 'cats_and_dogs_resnet18.pth')
print('model saved: cats_and_dogs_resnet18.pth')

Model saved as cats_and_dogs_resnet18.pth
